In [1]:
import os
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import pandas as pd

# Directory of unlabeled images
unlabeled_dir = 'dataset/images/images'

# Load a pretrained model (e.g., ResNet18)
model = models.resnet18(pretrained=True)
model.eval()  # Set to evaluation mode

# Image transformation for preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Function to classify unlabeled images and label them
def classify_and_label_unlabeled_images(unlabeled_dir):
    labeled_images = []
    with torch.no_grad():  # No need for gradients during inference
        for image_name in os.listdir(unlabeled_dir):
            image_path = os.path.join(unlabeled_dir, image_name)
            image = Image.open(image_path)
            image = transform(image).unsqueeze(0)  # Add batch dimension

            # Get the model's prediction
            output = model(image)
            _, predicted = torch.max(output, 1)  # Get the class with the highest probability

            # Add image and predicted label to the list
            labeled_images.append({'image': image_name, 'label': predicted.item()})
    
    return labeled_images

# Get the labeled images from the pretrained model
labeled_images = classify_and_label_unlabeled_images(unlabeled_dir)

# Convert labeled images to a DataFrame
labeled_df = pd.DataFrame(labeled_images)

# Save the DataFrame to a CSV file
labeled_df.to_csv('labeled_unlabeled_images.csv', index=False)

print(f"Labeled images saved to 'labeled_unlabeled_images.csv'.")


In [11]:
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

# Define a custom dataset class
class BrainTumorDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_files = os.listdir(root_dir)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.image_files[idx])
        image = Image.open(img_path).convert("RGB")  # Ensure RGB format
        if self.transform:
            image = self.transform(image)
        return image

# Define transformations (resize, normalize, etc.)
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),  # Converts to [C, H, W]
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize to [-1, 1]
])

# Load datasets
train_dataset = BrainTumorDataset(root_dir="processed_data/Training/processedimgs", transform=transform)
test_dataset = BrainTumorDataset(root_dir="processed_data/Testing/processedimgs", transform=transform)

# Create DataLoaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [3]:
import torch
import torch.nn as nn

class Generator(nn.Module):
    def __init__(self, latent_dim, img_channels, feature_map_size):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            # Input: [batch_size, latent_dim, 1, 1]
            nn.ConvTranspose2d(latent_dim, feature_map_size * 8, kernel_size=4, stride=1, padding=0),
            nn.BatchNorm2d(feature_map_size * 8),
            nn.ReLU(True),

            nn.ConvTranspose2d(feature_map_size * 8, feature_map_size * 4, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(feature_map_size * 4),
            nn.ReLU(True),

            nn.ConvTranspose2d(feature_map_size * 4, feature_map_size * 2, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(feature_map_size * 2),
            nn.ReLU(True),

            nn.ConvTranspose2d(feature_map_size * 2, img_channels, kernel_size=4, stride=2, padding=1),
            nn.Tanh()  # Output in range [-1, 1]
        )

    def forward(self, z):
        return self.model(z)

In [4]:
class Discriminator(nn.Module):
    def __init__(self, img_channels, feature_map_size):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            # Input: [batch_size, img_channels, height, width]
            nn.Conv2d(img_channels, feature_map_size, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(feature_map_size, feature_map_size * 2, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(feature_map_size * 2),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(feature_map_size * 2, feature_map_size * 4, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(feature_map_size * 4),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(feature_map_size * 4, 1, kernel_size=4, stride=1, padding=0),
            nn.Sigmoid()  # Output a single scalar value per image
        )

    def forward(self, x):
        return self.model(x).view(x.size(0), -1)  # Flatten to [batch_size, 1]

In [5]:
# Hyperparameters
latent_dim = 100
img_channels = 3  # Change to 1 if using grayscale images
feature_map_size = 64
lr = 0.0002
batch_size = 32
epochs = 50

# Initialize models
generator = Generator(latent_dim, img_channels, feature_map_size)
discriminator = Discriminator(img_channels, feature_map_size)

# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))

# Loss function
criterion = nn.BCELoss()

In [6]:
import os

output_dir = "processed_data/Training/processedimgs/pituitary_tumor"
print(f"Does the directory exist? {os.path.exists(output_dir)}")
print(f"Is the directory writable? {os.access(output_dir, os.W_OK)}")

Does the directory exist? True
Is the directory writable? True


In [7]:
import os

output_dir = "processed_data/Training/processedimgs/no_tumor"
print(f"Does the directory exist? {os.path.exists(output_dir)}")
print(f"Is the directory writable? {os.access(output_dir, os.W_OK)}")

Does the directory exist? True
Is the directory writable? True


In [8]:
output_dir = os.path.join("processed_data", "Training", "processedimgs", "pituitary_tumor")
os.makedirs(output_dir, exist_ok=True)

In [14]:
import os

output_dir = "processed_data/Training/processedimgs/meningioma_tumor"
if os.path.exists(output_dir) and len(os.listdir(output_dir)) > 0:
    print(f"Preprocessed data exists in {output_dir}")
else:
    print("Preprocessed data not found. Preprocessing is required.")

Preprocessed data exists in processed_data/Training/processedimgs/meningioma_tumor


In [15]:
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os

class BrainTumorDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_files = os.listdir(root_dir)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.image_files[idx])
        image = Image.open(img_path).convert("RGB")  # Ensure RGB format
        if self.transform:
            image = self.transform(image)
        return image

# Define transformations (resize, normalize, etc.)
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize to match preprocessing
    transforms.ToTensor(),          # Convert to tensor
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize to [-1, 1]
])

# Load datasets
train_dataset = BrainTumorDataset(root_dir="processed_data/Training/processedimgs", transform=transform)
test_dataset = BrainTumorDataset(root_dir="processed_data/Testing/processedimgs", transform=transform)

# Create DataLoaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [17]:
!pip install virtualenv

# Create a virtual environment named "venv"
virtualenv venv

# Activate the virtual environment
venv\Scripts\activate

SyntaxError: invalid syntax (2811859520.py, line 4)

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# Hyperparameters
latent_dim = 100
img_channels = 3  # Change to 1 if using grayscale images
feature_map_size = 64
lr = 0.0002
batch_size = 32
epochs = 50

# Generator and Discriminator (use your existing architectures)
generator = Generator(latent_dim, img_channels, feature_map_size)
discriminator = Discriminator(img_channels, feature_map_size)

# Optimizers
optimizer_G = optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))

# Loss function
criterion = nn.BCELoss()

# Training loop
for epoch in range(epochs):
    for i, real_imgs in enumerate(tqdm(train_loader)):
        # Real images and labels
        real_labels = torch.ones(real_imgs.size(0), 1)  # Shape: [batch_size, 1]
        fake_labels = torch.zeros(real_imgs.size(0), 1)  # Shape: [batch_size, 1]

        # Train Discriminator
        optimizer_D.zero_grad()
        real_loss = criterion(discriminator(real_imgs), real_labels)
        z = torch.randn(real_imgs.size(0), latent_dim, 1, 1)  # Random noise
        fake_imgs = generator(z)
        fake_loss = criterion(discriminator(fake_imgs.detach()), fake_labels)
        d_loss = real_loss + fake_loss
        d_loss.backward()
        optimizer_D.step()

        # Train Generator
        optimizer_G.zero_grad()
        g_loss = criterion(discriminator(fake_imgs), real_labels)  # Trick the discriminator
        g_loss.backward()
        optimizer_G.step()

    # Print progress
    print(f"[Epoch {epoch+1}/{epochs}] D Loss: {d_loss.item():.4f}, G Loss: {g_loss.item():.4f}")

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]


PermissionError: [Errno 13] Permission denied: 'processed_data/Training/processedimgs\\no_tumor'

In [13]:
from tqdm import tqdm

# Training loop
for epoch in range(epochs):
    for i, real_imgs in enumerate(tqdm(train_loader)):
        # Ensure real_imgs is on the correct device
        real_imgs = real_imgs
        
        # Dynamically create labels based on the current batch size
        real_labels = torch.ones(real_imgs.size(0), 1)  # Shape: [current_batch_size, 1]
        fake_labels = torch.zeros(real_imgs.size(0), 1)  # Shape: [current_batch_size, 1]

        # Train Discriminator
        optimizer_D.zero_grad()

        # Real images
        real_loss = criterion(discriminator(real_imgs), real_labels)

        # Fake images
        z = torch.randn(real_imgs.size(0), latent_dim, 1, 1)  # Random noise
        fake_imgs = generator(z)
        fake_loss = criterion(discriminator(fake_imgs.detach()), fake_labels)

        # Total discriminator loss
        d_loss = real_loss + fake_loss
        d_loss.backward()
        optimizer_D.step()

        # Train Generator
        optimizer_G.zero_grad()
        g_loss = criterion(discriminator(fake_imgs), real_labels)  # Trick the discriminator
        g_loss.backward()
        optimizer_G.step()

    # Print progress
    print(f"[Epoch {epoch+1}/{epochs}] D Loss: {d_loss.item():.4f}, G Loss: {g_loss.item():.4f}")

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]


PermissionError: [Errno 13] Permission denied: 'processed_data/Training/processedimgs\\meningioma_tumor'